In [24]:
import time
import torch
import torch.utils.data as data
from torch.autograd import Variable
from ssd import build_ssd
from layers.modules import MultiBoxLoss
from utils.augmentations import SSDAugmentation
from data import *

In [22]:
num_classes = 21
min_dim = 300
batch_size = 3
num_workers = 4
lr = 1e-4
gamma = 0.1


In [3]:
ssd_net = build_ssd('train', min_dim, num_classes)

In [4]:
vgg_weights = torch.load('weights/vgg16_reducedfc.pth')

In [5]:
ssd_net.vgg.load_state_dict(vgg_weights)

<All keys matched successfully>

In [20]:
optimizer = torch.optim.SGD(ssd_net.parameters(), lr=lr, momentum=0.9,
                          weight_decay=5e-4)

In [7]:
criterion = MultiBoxLoss(num_classes, 0.5, True, 0, True, 3, 0.5,
                            False, False)

In [11]:
dataset = VOCDetection(root='/home/vlad/park/ML/hw6/ssd.pytorch/data/VOCdevkit/',
                        transform=SSDAugmentation(min_dim, MEANS))

In [18]:
ssd_net.train()
# loss counters
loc_loss = 0
conf_loss = 0
epoch = 0
start_iter = 0
max_iter = 120000
step_index = 0
lr_steps = (80000, 100000, 120000)
print('Loading the dataset...')

epoch_size = len(dataset) // batch_size
print('Training SSD on:', dataset.name)

Loading the dataset...
Training SSD on: VOC2012


In [16]:
data_loader = data.DataLoader(dataset, batch_size,
                                num_workers=num_workers,
                                shuffle=True, collate_fn=detection_collate,
                                pin_memory=True)

In [21]:
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [26]:
batch_iterator = iter(data_loader)
for iteration in range(start_iter, max_iter):

    if iteration in lr_steps:
        step_index += 1
        adjust_learning_rate(optimizer, gamma, step_index)

    # load train data
    images, targets = next(batch_iterator)


    images = Variable(images)
    targets = [Variable(ann, volatile=True) for ann in targets]

    # forward
    t0 = time.time()
    out = ssd_net(images)
    # backprop
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, targets)
    loss = loss_l + loss_c
    loss.backward()
    optimizer.step()
    t1 = time.time()
    loc_loss += loss_l.data.item()
    conf_loss += loss_c.data.item()

    if iteration % 10 == 0:
            print('timer: %.4f sec.' % (t1 - t0))
            print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.data.item()), end=' ')


    if iteration != 0 and iteration % 5000 == 0:
        print('Saving state, iter:', iteration)
        torch.save(ssd_net.state_dict(), 'weights/ssd300_COCO_' +
                    repr(iteration) + '.pth')
torch.save(ssd_net.state_dict(), 'weights/VOC.pth')

/tmp/ipykernel_37642/2011913417.py:13: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  targets = [Variable(ann, volatile=True) for ann in targets]
/home/vlad/.venv/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


timer: 2.0179 sec.
iter 0 || Loss: 18.9840 || timer: 1.9985 sec.
iter 10 || Loss: 15.7797 || timer: 1.9497 sec.
iter 20 || Loss: 14.8692 || timer: 1.7450 sec.
iter 30 || Loss: 15.7853 || timer: 1.8495 sec.
iter 40 || Loss: 15.1252 || timer: 1.9620 sec.
iter 50 || Loss: 14.9281 || timer: 1.9104 sec.
iter 60 || Loss: 14.7227 || timer: 1.7760 sec.
iter 70 || Loss: 14.4776 || 

KeyboardInterrupt: 